In [8]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import bleach

In [9]:
def getPersonInfo(nid):
    url_template = "https://eds.nd.edu/cgi-bin/nd_ldap_search.pl?displayformat=/ndcso_tmp.html&ldapfilter=uid={nid}"
    url = url_template.format(nid=nid)
    try:
        r = requests.get(url)
    except requests.exceptions.ConnectionError:
        return 
    
    soup = BeautifulSoup(r.text)

    if(soup.find(text="Found 0 entries.") != None):
        return
    else:
        netid = soup.find(text="NetID:")
        b_tag = netid.parent
        td_tag = b_tag.parent
        next_td_tag = td_tag.findNext('td')
        next2_td_tag = next_td_tag.findNext('td')
        #print next2_td_tag.contents[0]
        cleannetid = bleach.clean(next2_td_tag, tags=[], strip=True)
        valnetid=[]
        valnetid.append(cleannetid)

        dept = soup.find(text="Department:")
        b_tag = dept.parent
        td_tag = b_tag.parent
        next_td_tag = td_tag.findNext('td')
        next2_td_tag = next_td_tag.findNext('td')
        cleandept = bleach.clean(next2_td_tag, tags=[], strip=True)
        valdept=[]
        valdept.append(cleandept)

        title = soup.find(text="Title:")
        b_tag = title.parent
        td_tag = b_tag.parent
        next_td_tag = td_tag.findNext('td')
        next2_td_tag = next_td_tag.findNext('td')
        cleantitle = bleach.clean(next2_td_tag, tags=[], strip=True)
        valtitle=[]
        valtitle.append(cleantitle)

        affil = soup.find(text="Affiliation:")
        b_tag = affil.parent
        td_tag = b_tag.parent
        next_td_tag = td_tag.findNext('td')
        next2_td_tag = next_td_tag.findNext('td')
        cleanaffil = bleach.clean(next2_td_tag, tags=[], strip=True)
        valaffil=[]
        valaffil.append(cleanaffil)

        curric = soup.find(text="Curriculum:")
        b_tag = curric.parent
        td_tag = b_tag.parent
        next_td_tag = td_tag.findNext('td')
        next2_td_tag = next_td_tag.findNext('td')
        cleancurric = bleach.clean(next2_td_tag, tags=[], strip=True)
        valcurric=[]
        valcurric.append(cleancurric)

        email = soup.find(text="Preferred email:")
        b_tag = email.parent
        td_tag = b_tag.parent
        next_td_tag = td_tag.findNext('td')
        next2_td_tag = next_td_tag.findNext('td')
        cleanemail = bleach.clean(next2_td_tag, tags=[], strip=True)
        valemail=[]
        valemail.append(cleanemail)

        df = pd.DataFrame({'netid':valnetid, 'dept':valdept, 'title':valtitle, 'affil':valaffil, 'curric':valcurric, 'email':valemail})
        df=df[['netid', 'email', 'affil','dept', 'title','curric']]
        return df


In [10]:
persons_df=pd.DataFrame()

In [11]:
netidlist=pd.read_csv('netids.csv',sep='\n')

In [12]:
# for who in ['jng2','slee45','edziadul','jthurk','jvecchio','eschnurr']:
for index, row in netidlist.iterrows():
    who = row['netid']
    pers_df = getPersonInfo(who)
    persons_df = persons_df.append(pers_df)

In [13]:
persons_df.reset_index(drop=True)

,netid,email,affil,dept,title,curric
0,jng2,james.ng@nd.edu,Faculty,Hesburgh Libraries,Assistant Librarian,
1,slee45,,Student,,,Graduate Business School


In [14]:
persons_df.to_csv('ndpersonsinfo.csv',index=False)